#### Test the downloaded file

# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.webservice import LocalWebservice

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import urllib.request
from azureml.core.model import Model


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


# Create or Attach an AmlCompute cluster

In [2]:
ws = Workspace.from_config()
experiment_name = 'capstone_hyperdrive_exp'

experiment=Experiment(workspace=ws, name=experiment_name)

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-189674
aml-quickstarts-189674
southcentralus
6971f5ac-8af1-446e-8034-05acea24681f


In [3]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "hyperdrive-com-clst" #automl-com-clst #automl-com-clst

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key="Maternal_Health_Risk_Data_Set" 
description_text = "UCI machine Learning Maternal Health Risk Data Set"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
#https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv
#https://github.com/ShubraChowdhury/AzureMLCapstone/blob/master/starter_file/Maternal_Health_Risk_Data_Set.csv

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate
count,1014.000000,1014.000000,1014.000000,1014.000000,1011.000000,1014.000000
mean,29.871795,113.198225,76.460552,8.725986,98.665678,74.301775
std,13.474386,18.403913,13.885796,3.293532,1.373367,8.088702
min,10.000000,70.000000,49.000000,6.000000,98.000000,7.000000
25%,19.000000,100.000000,65.000000,6.900000,98.000000,70.000000
50%,26.000000,120.000000,80.000000,7.500000,98.000000,76.000000
75%,39.000000,120.000000,90.000000,8.000000,98.000000,80.000000
max,70.000000,160.000000,100.000000,19.000000,103.000000,90.000000


Review the Dataset Result

In [5]:
dataset.take(5).to_pandas_dataframe()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98,86,high risk
1,35,140,90,13.0,98,70,high risk
2,29,90,70,8.0,100,80,high risk
3,30,140,85,7.0,98,70,high risk
4,35,120,60,6.1,98,76,low risk


In [6]:
from sklearn.model_selection import train_test_split

TRAIN_d, TEST_d = train_test_split(df, test_size=0.2, random_state=24, shuffle=True)
TRAIN_d.to_csv('train.csv',index = False)

ds_new = ws.get_default_datastore()
ds_new.upload_files(files = ['./train.csv'])

train_data = Dataset.Tabular.from_delimited_files(path=[(ds_new,('train.csv'))])

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [7]:
from sklearn.model_selection import train_test_split
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
#BanditPolicy(evaluation_interval=1, slack_factor=None, slack_amount=None, delay_evaluation=0)
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
#solver{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, default=’lbfgs’
ps = RandomParameterSampling(
    {
"--C" : choice(0.01,0.1,1) ,     
"--max_iter" : choice(20,40,70,100,150),
#"--solver" :  choice('newton-cg','lbfgs','liblinear','sag', 'saga')        
}
)  


#TODO: Create your estimator and hyperdrive config


if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='project_environment.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#SKLearn(source_directory, *, compute_target=None, vm_size=None, vm_priority=None, entry_script=None, script_params=None, use_docker=True, custom_docker_image=None, image_registry_details=None, user_managed=False, conda_packages=None, pip_packages=None, conda_dependencies_file_path=None, pip_requirements_file_path=None, conda_dependencies_file=None, pip_requirements_file=None, environment_variables=None, environment_definition=None, inputs=None, shm_size=None, resume_from=None, max_run_duration_seconds=None, framework_version=None, _enable_optimized_mode=False, _disable_validation=True, _show_lint_warnings=False, _show_package_warnings=False)
skl_estimator = SKLearn(source_directory="./",entry_script='train.py',compute_target=amlcompute_cluster_name)


hyperdrive_run_config =  HyperDriveConfig(
        hyperparameter_sampling=ps,
        primary_metric_name='Accuracy',
        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
        max_total_runs = 20, 
        max_concurrent_runs =4,
        policy=early_termination_policy,
        estimator=skl_estimator
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [8]:
#TODO: Submit your experiment

hyperdrive_run=experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_dacbae8c-454e-455d-a050-0ce6edfd9cd3
Web View: https://ml.azure.com/runs/HD_dacbae8c-454e-455d-a050-0ce6edfd9cd3?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-189674/workspaces/quick-starts-ws-189674&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-03-22T19:57:36.374386][API][INFO]Experiment created<END>\n""<START>[2022-03-22T19:57:37.140097][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-03-22T19:57:37.767002][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2022-03-22T19:58:07.511725][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-03-22T19:58:07.772982][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2022-03-22T19:58:36.8026834Z][SCHEDULER][INFO]Scheduling job, id='

{'runId': 'HD_dacbae8c-454e-455d-a050-0ce6edfd9cd3',
 'target': 'hyperdrive-com-clst',
 'status': 'Completed',
 'startTimeUtc': '2022-03-22T19:57:36.168031Z',
 'endTimeUtc': '2022-03-22T20:16:39.817352Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '821b1c93-7cb3-45b2-8045-ac578854df06',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1068-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '15',
  'score': '0.6059113300492611',
  'best_child_run_id': 'HD_dacbae8c-454e-455d-a050-0ce6edfd9cd3_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg189674.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_dacbae8c-454e-455d-a050-0ce6edfd9cd3/a

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [11]:
### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics() 

print("Best Run ID :", best_run.id)
print("\n Accuracy :", best_run_metrics['Accuracy'])

print("\n best run file names :",best_run.get_file_names())
print("\n best run details ", best_run.get_details())
print("\n best run metrics :",best_run_metrics)


Best Run ID : HD_dacbae8c-454e-455d-a050-0ce6edfd9cd3_0

 Accuracy : 0.6059113300492611

 best run file names : ['azureml-logs/20_image_build_log.txt', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'outputs/model.joblib', 'outputs/model.pkl', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']

 best run details  {'runId': 'HD_dacbae8c-454e-455d-a050-0ce6edfd9cd3_0', 'target': 'hyperdrive-com-clst', 'status': 'Completed', 'startTimeUtc': '2022-03-22T20:08:48.305101Z', 'endTimeUtc': '2022-03-22T20:10:00.060895Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '821b1c93-7cb3-45b2-8045-ac578854df06', 'ProcessInfoFile': 'azureml-logs/proc

In [12]:
best_run_metrics

{'Regularization Strength:': 0.1,
 'Max iterations:': 150,
 'Accuracy': 0.6059113300492611}

In [13]:
print("Best Run Status :", best_run.status)

Best Run Status : Completed


In [14]:
print("Best Run Properties :", best_run.properties)

Best Run Properties : {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '821b1c93-7cb3-45b2-8045-ac578854df06', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
model = best_run.register_model(model_name='Maternal_Health_Risk_HyperDrive_model', 
model_path='outputs/model.joblib', tags={"Method" : "HyperDrive"},
properties={"Accuracy": best_run_metrics['Accuracy']})


In [16]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice

inference_config = InferenceConfig(entry_script="scorehyperdrive.py", environment = sklearn_env) #sklearn_env

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 4,enable_app_insights = True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
service=Model.deploy(workspace=ws,
                    name="meternal-health-hyper-deploy-ser",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

In [18]:
print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')



TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
scoring_uri = service.scoring_uri

data = {
  "data": [
    {
      "Age": 25,
      "SystolicBP":130 ,
      "DiastolicBP":80 ,
      "BS": 15,
      "BodyTemp": 98,
      "HeartRate": 86
          }
  ]
}
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

In [19]:
print(service.get_logs())

In [20]:
service.delete()

In [21]:
model.delete()
amlcompute_cluster_name.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

